### ML PYspark

The first thing to do is to create a .env file in the root of the directory. Add to the file the following two varibles 
ACCESS_KEY, ACCESS_SECRET. 
Check for more detailed explanation here: [dotenv]("https://pypi.org/project/python-dotenv/), he explains how the .env should look like. After that, the variables are add to the os.environ and can be access as a simple dict structure

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as spark_sum, when, input_file_name
from functools import reduce
import sys
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler
import os
from pyspark.ml.classification import LogisticRegression
from dotenv import load_dotenv

In [2]:
import random
## load .env
load_dotenv()


True

## Console Login

The following classes are to handle the spark on the AWS 

In [3]:
from src.s3handler import Sparker, PreProcessing, FeatureEngineering

In [4]:
## Initialize the class
spark = Sparker(os.environ['ACCESS_KEY'],os.environ['ACCESS_SECRET'])

## local session
spark._create_local_session()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/06 06:33:41 WARN Utils: Your hostname, DESKTOP-95V5VE8, resolves to a loopback address: 127.0.1.1; using 172.30.46.218 instead (on interface eth0)
25/11/06 06:33:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/manecomaneca/venv/spark/.venv/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/manecomaneca/.ivy2.5.2/cache
The jars for the packages stored in: /home/manecomaneca/.ivy2.5.2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e42d0f56-35b1-4bb6-bd95-ad798c79dbd4;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution 

## Read parquet

In [5]:
parquet_cols = ["xyz","Intensity","Classification","Red","Green","Blue","Infrared","ReturnNumber","NumberOfReturns"]

## Read the parquet and stored it 
df = spark.read_parquet("ubs-datasets",
                    "FRACTAL/data/train/TRAIN-0436_6399-002955400.parquet",
                    read_all=False) \
                    .select(*parquet_cols)

# # Read the list of parquet files
# list_s3 = ["FRACTAL/data/train/TRAIN-1200_6136-008972557.parquet", "FRACTAL/data/train/TRAIN-0436_6399-002955400.parquet"]
# df = spark.read_parquet("ubs-datasets",
#                     list_s3,
#                     read_all=False)

Reading from: ['s3a://ubs-datasets/FRACTAL/data/train/TRAIN-0436_6399-002955400.parquet']


In [ ]:
df.printSchema()
print(f"Number of rows: {df.count()}")

root
 |-- xyz: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- Intensity: integer (nullable = true)
 |-- Classification: short (nullable = true)
 |-- Red: integer (nullable = true)
 |-- Green: integer (nullable = true)
 |-- Blue: integer (nullable = true)
 |-- Infrared: integer (nullable = true)
 |-- ReturnNumber: short (nullable = true)
 |-- NumberOfReturns: short (nullable = true)



## Preprocessing & Feature Engineering

In [6]:
df.columns

['xyz',
 'Intensity',
 'Classification',
 'Red',
 'Green',
 'Blue',
 'Infrared',
 'ReturnNumber',
 'NumberOfReturns']

In [6]:
preprocessing = PreProcessing(df)
df = preprocessing.split_xyz()

# ## feature engineering
engfeature = FeatureEngineering(df)
df = engfeature.apply_all()

In [8]:
df.columns

['Intensity',
 'Classification',
 'Red',
 'Green',
 'Blue',
 'Infrared',
 'ReturnNumber',
 'NumberOfReturns',
 'x',
 'y',
 'z',
 'height_above_ground',
 'local_density',
 'local_z_std',
 'local_z_range',
 'roughness',
 'return_ratio',
 'is_single_return',
 'is_last_return',
 'ndvi',
 'green_red_ratio',
 'ndwi']

In [9]:
from pyspark.sql.functions import col, sum as _sum
null_counts = df.select([
    _sum(col(c).isNull().cast("int")).alias(c) 
    for c in feature_cols
])
null_counts.show()

+---------+---+-----+----+--------+------------+---------------+---+---+---+-------------------+-------------+-----------+-------------+---------+------------+----------------+--------------+----+---------------+----+
|Intensity|Red|Green|Blue|Infrared|ReturnNumber|NumberOfReturns|  x|  y|  z|height_above_ground|local_density|local_z_std|local_z_range|roughness|return_ratio|is_single_return|is_last_return|ndvi|green_red_ratio|ndwi|
+---------+---+-----+----+--------+------------+---------------+---+---+---+-------------------+-------------+-----------+-------------+---------+------------+----------------+--------------+----+---------------+----+
|        0|  0|    0|   0|       0|           0|              0|  0|  0|  0|                  0|            0|          0|            0|        0|           0|               0|             0|   0|              0|   0|
+---------+---+-----+----+--------+------------+---------------+---+---+---+-------------------+-------------+-----------+------

### Load | Models
Prepare the variable for the models

In [8]:
feature_cols = df.drop("Classification").columns  
assembler = VectorAssembler(inputCols=feature_cols,
                            outputCol="features",
                           # handleInvalid="skip" 
                           ) 

scaler = StandardScaler(inputCol="features", outputCol="scaled_features")


In [10]:
# 3. Define model
lr = LogisticRegression(
    featuresCol="scaled_features",
    labelCol="Classification",
    maxIter=10
)

### Pipeline

In [11]:
pipeline = Pipeline(stages=[assembler, scaler, lr])

### Train 

In [ ]:
## Import taskmetrics to see how the model is performing
taskmetrics = TaskMetrics(sparker.spark)

In [12]:
model = pipeline.fit(df)

In [14]:
model.transform(df).show(2)

+---------+--------------+-----+-----+-----+--------+------------+---------------+----------+-----------+------+--------------------+-------------+-------------------+-----------------+-------------------+------------+----------------+--------------+-------------------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+----------+
|Intensity|Classification|  Red|Green| Blue|Infrared|ReturnNumber|NumberOfReturns|         x|          y|     z| height_above_ground|local_density|        local_z_std|    local_z_range|          roughness|return_ratio|is_single_return|is_last_return|               ndvi|   green_red_ratio|              ndwi|            features|     scaled_features|       rawPrediction|         probability|prediction|
+---------+--------------+-----+-----+-----+--------+------------+---------------+----------+-----------+------+--------------------+-------------+-------------------+-----------------+-------

## Inference

In [16]:
df_test = spark.read_parquet("ubs-datasets",
                    "FRACTAL/data/train/TRAIN-1200_6136-008972557.parquet",
                    read_all=False) \
                    .select(*parquet_cols)
                    
preprocessing = PreProcessing(df_test)
df_test = preprocessing.split_xyz()

# ## feature engineering
engfeature = FeatureEngineering(df_test)
df_test = engfeature.apply_all()

#feature_cols = df_test.drop("Classification").columns  

predictions = model.transform(df_test)

Reading from: ['s3a://ubs-datasets/FRACTAL/data/train/TRAIN-1200_6136-008972557.parquet']


In [17]:
test_cols = ["Classification","Prediction","Probability"]
predictions.select(test_cols).show(2)

+--------------+----------+--------------------+
|Classification|Prediction|         Probability|
+--------------+----------+--------------------+
|             4|       2.0|[0.0,0.0,1.0,0.0,...|
|             4|       2.0|[0.0,0.0,1.0,0.0,...|
+--------------+----------+--------------------+
only showing top 2 rows


## Evaluation

In [19]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [20]:
evaluator = MulticlassClassificationEvaluator(
    labelCol = 'Classification',
    predictionCol = 'Prediction',
    metricName = 'accuracy'
)

accuracy = evaluator.evaluate(predictions)
print(f"Test Accuracy: {accuracy :.3f}")

Test Accuracy: 0.345


In [21]:
df.count()

90090

In [22]:
df_test.count()

209895